## Naive Bayes EDA and model tweaking

This is a follow up to the Udacity base analysis of the SMS Spam dataset.  Looking into methods of increasing the accuracy of the model.

In [9]:
import numpy
import pandas as pd
import pprint
from collections import Counter
pp = pprint.PrettyPrinter(indent = 4)


In [4]:
# Dataset from - https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
df = pd.read_table('smsspamcollection/SMSSpamCollection',
                   sep='\t', 
                   header=None, 
                   names=['label', 'sms_message'])

# Output printing out first 5 columns
df.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Storing column names as variables and converting label(outcome) to numeric.

In [6]:
outcome = 'label'
features = ['sms_message']

df[outcome] = df.label.map({'ham':0, 'spam':1})
print(df.shape)
df.head() # returns (rows, columns)

(5572, 2)


,label,sms_message
0,NaN,"Go until jurong point, crazy.. Available only ..."
1,NaN,Ok lar... Joking wif u oni...
2,NaN,Free entry in 2 a wkly comp to win FA Cup fina...
3,NaN,U dun say so early hor... U c already then say...
4,NaN,"Nah I don't think he goes to usf, he lives aro..."


### Function storing

In [10]:
def condLower(sentence):
    return ' '.join([i.lower() if i != i.upper() else i for i in sentence.split(' ')])

## Getting CountVectorizer from sklearn

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer()

print(count_vector)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [ ]:
count_vector.fit(documents)
count_vector.get_feature_names()